# MGO Model to Create MGO Feature Service on Kent County REST Server

<b> Notes: </b>
<ul><li>Adopted from Model.py script and MGO model.</li></ul>
<b>TO DO:</b>
<ul><li> Find Original Model on Danielle's Computer.</li>
<li> Need to run the weekly script to make sure all the environmental data is current.</li>



</ul>


In [1]:
#Use arcNew environment with this notebook.
#Current Location: C:\ProgramData\Anaconda3\envs\arcNew\python.exe
import arcpy as arc
import os
from shutil import copy2
import pandas as pd
import numpy as np
from arcgis import features
from datetime import datetime as dt
from pathlib import Path

In [2]:
arc.env.overwriteOutput = True
arc.env.outputZFlag = 'Disabled' #To remove z data from parcel fabric due to it being a polygonZ 
arc.env.qualifiedFieldNames = False
now = dt.now()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'
# print(netDB)

C:\Users\MKinnaman\GIS\Processing already exists.
C:\Users\MKinnaman\GIS\Review already exists.


### Only use with notebook

In [3]:
pd.options.display.max_columns = 40

## Create File GeoDatabase and Feature Datasets

### Create Directories for MGO Model GDB, for processing, and for Review.

In [4]:
#Create Folders for MGO Model
mFolder = [f for f in lPath if f.name == 'Processing'][0]
mProcessing = mFolder / 'MGO_Model' / f'{dStr}'
if mProcessing.exists() == True:
    print(f'{mProcessing} already exist.')
else:
    mProcessing.mkdir(parents=True)
    print(f'Created {mProcessing}.')

mFR = [f for f in lPath if f.name == 'Review'][0]
mReview = mFR / 'MGO_Model' / f'{dStr}'
if mReview.exists() == True:
    print(f'{mReview} already exist')
else:
    mReview.mkdir(parents=True)
    print(f'Created {mReview}')

Created C:\Users\MKinnaman\GIS\Processing\MGO_Model\05_07.
Created C:\Users\MKinnaman\GIS\Review\MGO_Model\05_07


In [5]:
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
sr = arc.Describe(f'{iE}').spatialReference
outGDB = gisPath / mFolder / f'Data_{mStr}.gdb'
# dsA = gisPath / mFolder / f'{outGDB}.gdb'
locGDB = outGDB / f'Daily_{dStr}'
mgoGDB = outGDB / f'MGO_{dStr}'
if arc.Exists(f'{outGDB}'):    
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(mFolder, f'{outGDB.name}')
    print(f'Created File GeoDatabase at {outGDB.parent}')

time.sleep(2)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

time.sleep(2)

if arc.Exists(f'{mgoGDB}'):
    print(f'{mgoGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{mgoGDB.parent}', f'{mgoGDB.name}', sr)
    print(f'{mgoGDB.name} Dataset has been created')

GDB already exists.
Daily_05_07 already exists
MGO_05_07 Dataset has been created


### Export Needed Feature Classes to Local GDB

In [6]:
dbTaxParcels = "PROD.GISADMIN.TaxParcels"
dbZoningPoly = "PROD.PLANNINGADMIN.OfficialZoning"
dbApz = "PROD.GISADMIN.APZ"
dbAgDist = "PROD.GISADMIN.Ag_Districts"
dbAPFO = "PROD.PLANNINGADMIN.APFO"
dbNoiseContour = "PROD.GISADMIN.Noise_Contour_Area"
dbAirportZone = "PROD.GISADMIN.Airport_Zones_onlyKentCounty"
dbAgEasement = "PROD.GISADMIN.Ag_District_Easement"
dbAgExpansion = "PROD.GISADMIN.AgDistrictExpansion"
dbAdd = 'PROD.ADDRESSINGADMIN.Addressing'
dbSufP = 'PROD.GISADMIN.Suffix_Parcels'

In [7]:
#create paths to the data instead of using env.workspace makes it so that I can use arc Walk
arc.env.workspace = f'{iE.parent}'

mgoList = [dbTaxParcels, dbZoningPoly, dbApz, dbAgDist, dbAPFO, dbNoiseContour, dbAirportZone, dbAgEasement, dbAgExpansion]
fcList = [dbAdd, dbSufP]
mParcels = []
cParcels = []
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in mgoList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}'
            if arc.Exists(f'{mgoGDB / b}'):
                mParcels.append(mgoGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.FeatureClassToFeatureClass_conversion(f, f'{mgoGDB}', f'{b}')
                mParcels.append(mgoGDB.joinpath(b))
                print (f'{b} has been copied')

time.sleep(2)

for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in fcList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}'
            if arc.Exists(f'{locGDB / b}'):
                cParcels.append(locGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.FeatureClassToFeatureClass_conversion(f, f'{locGDB}', f'{b}')
                cParcels.append(locGDB.joinpath(b))
                print (f'{b} has been copied')

APZ_05_07 has been copied
Noise_Contour_Area_05_07 has been copied
Airport_Zones_onlyKentCounty_05_07 has been copied
AgDistrictExpansion_05_07 has been copied
Ag_Districts_05_07 has been copied
Ag_District_Easement_05_07 has been copied
OfficialZoning_05_07 has been copied
APFO_05_07 has been copied
TaxParcels_05_07 has been copied
Addressing_05_07 already exists
Suffix_Parcels_05_07 already exists


### Works with Parcel Fabric

In [8]:
#currently using env.workspace to move parcelfabric needs to be changed to use path
pfParcels = 'PROD.MAPPINGADMIN.ParcelFabric_Parcels'
exp = "TYPE = 7 AND Historical = 0"
a = pfParcels.split('.')[-1]
b = f'{a}_{dStr}'
# print(f'{iEPath[1]}')
if arc.Exists(f'{locGDB / b}'):
    cParcels.append(locGDB.joinpath(b))
    print(f'{b} already exists')
else:
    print(f'Copying Parcel Fabric to {locGDB}')
    #print(a)
    # g = f'{f}'
    #print(g)
    arc.FeatureClassToFeatureClass_conversion(pfParcels, f'{locGDB}', b, where_clause=expr)
    cParcels.append(locGDB.joinpath(b))
    print(f'Finished Copying {b} to {locGDB}')

ParcelFabric_Parcels_05_07 already exists


### Create Spatial Joins with the exported Feature Classes.

Create Addressing DataFrame and create new address field for MGO.

In [ ]:
# Get the Addressing Feature Clas from the MGO GDB
gdbAdd = [f for f in cParcels if f.name.startswith('Addressing') == True][0]

df_add = features.GeoAccessor.from_featureclass(gdbAdd)
print('Address DataFrame created.')

In [ ]:
# df_add['STS'].replace('None', " ")
# df_add.loc[df_add.STS == None, :]
# df_add.isnull().sum()
df_add['STS'].fillna(np.nan, inplace= True)
df_add['STS'].replace(' ', '', inplace=True)
df_add['STS'].replace(np.nan, '', inplace=True)
# df_add['STS'].unique()
# df_add.head()

In [ ]:
aType = df_add.STS.unique()
# print(sType)
for d in aType:
    if d.upper() == 'ROAD':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'RD').strip())
    elif d.upper() == 'DRIVE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'DR').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'DR').strip())
    elif d.upper() == 'BOULEVARD':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'BLVD').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'BLVD').strip())
    elif d.upper() == 'LANE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'LN').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'LN').strip())
    elif d.upper() == 'COURT':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'CT').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CT').strip())
    elif d.upper() == 'CIRCLE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'CIR').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CIR').strip())
    elif d.upper() == 'STREET':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'ST').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'ST').strip())
    elif d.upper() == 'HIGHWAY':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'HWY').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'HWY').strip())
    elif d.upper() == 'AVENUE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'AVE').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'AVE').strip())
    elif d.upper() == 'PLACE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'PL').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'PL').strip())
    else:
        df_add.STS.str.strip()
# print(df_add.STS.unique())

In [ ]:
df_add.fillna(np.nan, inplace= True)
# df_add.replace(' ', '', inplace=True)
df_add.replace(np.nan, '', inplace=True)
# print(df_add.columns)
# print(df_add.STS.value_counts())

In [ ]:
df_add.SAN = df_add.SAN.astype('str',errors='ignore')
df_add.SAN = df_add.SAN.apply(lambda x: x.split('.')[0])
df_add['FAddress'] = df_add[['SAN', 'PRD', 'STN', 'STS', 'POD']].values.tolist()
# df_add.FAddress.values
df_add['FooAddress'] = df_add['FAddress'].apply(' '.join)
df_add['Address911'] = df_add['FooAddress'].apply(lambda x: ' '.join(x.split()))
# df_add

In [ ]:
# df_add.FAddress = df_add.FoAddress
kColumns = ['Address911', 'SHAPE']
# print([x for x in kColumns if x not in kColumns])
df_add.drop(columns=[x for x in df_add if x not in kColumns], inplace=True)
# df_add

Create Tax Parcels DataFrame and join with Addressing

In [ ]:
gdbTax = [f for f in mParcels if f.name.startswith('TaxParcels') == True][0]
fTax = ['Name', 'Pride', 'Lot_Number', 'LOCATION', 'OWNERNAME', 'SECONDARYOWNER', 'MAILINGADDRESS', 'MAILINGADDRESS2', 'OWNERCITY', 'OWNERSTATE', 'OWNERZIP']
arc.env.workspace = f'{mgoGDB}'
dTaxC = [f.name for f in arc.ListFields(f'{gdbTax.name}') if f.name not in fTax]

arc.DeleteField_management(f'{gdbTax}', dTaxC)
arc.SpatialJoin_analysis(f'{gdbTax}', f'{gdbAdd}', f'{mgoGDB / "Add_Join"}')

# tax_df = features.GeoAccessor.from_featureclass(gdbTax, fields=fTax)
# print('TaxParcels DataFrame Created')

In [ ]:
# Remove Null Values from the SHAPE Column
nParcels = mReview / 'Null_Parcels'
tax_df.loc[tax_df.SHAPE.notnull()].to_csv(nParcels, index=False)
df_tax = tax_df.loc[tax_df.SHAPE.notnull()]

In [ ]:
df_join = df_tax.spatial.join(df_add, how='left')
# df_join

In [ ]:
df_join.Name.value_counts()

In [ ]:
df_join.loc[df_join['Address911'] == '1232 MEDIA RD']

In [ ]:
df_join.Name == '1-17-01900-01-0102-00001'

In [ ]:
at_df = df_join.loc[df_join.index_right.notnull()]
at_df.shape

In [ ]:
arc.SpatialJoin_analysis(f'{gdbTax}', f'{gdbAdd}', f'{mgoGDB / "Address_Join"}')
arc.GetCount_management(f'{mgoGDB / "Address_Join"}')

In [ ]:
df_join.drop(columns='index_right', inplace=True)

Join aiport zones with addressing join.

In [ ]:
gdbAirZ = [f for f in mParcels if f.name.startswith('Airport_Zones') == True][0]

az_df = features.GeoAccessor.from_featureclass(gdbAirZ, fields=['Airport'])

In [ ]:
aza_join = df_join.spatial.join(az_df, how='left')
aza_join

In [114]:
aza_join.spatial.full_extent

(170384.875, 92243.48900000006, 208368.15479999967, 151648.81670000032)

In [ ]:
gdbTax = [f for f in mParcels if f.name.startswith('TaxParcels') == True][0]

In [ ]:
gdbTax = [f for f in mParcels if f.name.startswith('TaxParcels') == True][0]